<a href="https://colab.research.google.com/github/ravilladivyaUNT/Divya_INFO5731_-Spring2022/blob/main/INFO5731_Assignment_Four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA and LSA. The following information should be reported:

(1) Features (top n-gram phrases) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster. 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Write your code here

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyldavis
import pyLDAvis
import gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings

data = pd.read_csv("/content/gdrive/My Drive/data.csv")

import nltk
nltk.download('stopwords')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

reviews = list(data['clean_text'].values)

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

reviews_words = list(sent_to_words(reviews))

print(reviews_words[:1])


You should consider upgrading via the 'C:\Users\maganti\anaconda3\python.exe -m pip install --upgrade pip' command.


C:\Users\maganti\anaconda3\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\maganti\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\maganti\anaconda3\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\maganti\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\maganti\anaconda3\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version ins

[['amazing', 'movie', 'marvel', 'studios', 'hyped', 'movie', 'since', 'first', 'teaser', 'movie', 'actually', 'exceedes', 'expectations', 'marvel', 'showing', 'marvelous', 'magic', 'since', 'ironman', 'shangchi', 'another', 'diamond', 'crown', 'end', 'mcus', 'phase', 'fans', 'eagerly', 'waiting', 'big', 'movie', 'black', 'widow', 'mark', 'eternals', 'also', 'disappointed', 'majority', 'fans', 'changchi', 'filled', 'gap', 'action', 'scenes', 'fresh', 'engaging', 'martial', 'arts', 'combined', 'chinese', 'mythological', 'powers', 'makes', 'action', 'scenes', 'interesting', 'character', 'arc', 'story', 'building', 'satisfying', 'rushed', 'slow', 'paced', 'whole', 'movie', 'engaging', 'till', 'end', 'would', 'love', 'see', 'another', 'movie', 'franchise']]


# 1.) 

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(reviews_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[reviews_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[reviews_words[0]]])


['amazing', 'movie', 'marvel', 'studios', 'hyped', 'movie', 'since', 'first', 'teaser', 'movie', 'actually', 'exceedes', 'expectations', 'marvel', 'showing', 'marvelous', 'magic', 'since', 'ironman', 'shangchi', 'another', 'diamond', 'crown', 'end', 'mcus', 'phase', 'fans', 'eagerly', 'waiting', 'big', 'movie', 'black_widow', 'mark', 'eternals', 'also', 'disappointed', 'majority', 'fans', 'changchi', 'filled', 'gap', 'action', 'scenes', 'fresh', 'engaging', 'martial_arts', 'combined', 'chinese', 'mythological', 'powers', 'makes', 'action', 'scenes', 'interesting', 'character', 'arc', 'story', 'building', 'satisfying', 'rushed', 'slow', 'paced', 'whole', 'movie', 'engaging', 'till', 'end', 'would', 'love', 'see', 'another', 'movie', 'franchise']


. We used bigrmas and trigrams phrases from the corpus as features

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


reviews_words_nostops = remove_stopwords(reviews_words)

# Form Bigrams
reviews_words_bigrams = make_bigrams(reviews_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
reviews_lemmatized = lemmatization(reviews_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(reviews_lemmatized[:1])

[['amazing', 'movie', 'marvel', 'studio', 'hype', 'movie', 'first', 'teaser', 'movie', 'actually', 'exceed', 'expectation', 'marvel', 'show', 'marvelous', 'magic', 'ironman', 'shangchi', 'diamond', 'crown', 'end', 'mcus', 'phase', 'fan', 'eagerly', 'wait', 'big', 'movie', 'mark', 'eternal', 'also', 'disappoint', 'majority', 'fan', 'changchi', 'fill', 'gap', 'action', 'scene', 'fresh', 'engage', 'martial_art', 'combine', 'chinese', 'mythological', 'power', 'make', 'action', 'scene', 'interesting', 'character', 'arc', 'story', 'building', 'satisfy', 'rush', 'slow', 'pace', 'whole', 'movie', 'engage', 'end', 'love', 'see', 'movie', 'franchise']]


# 2.) Top 10 Clusters for topic modelling

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(reviews_lemmatized)

# Create Corpus
texts = reviews_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])
print()

# builiding ldamodel
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# printing the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 2), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 6), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)]]

[(0,
  '0.030*"stand" + 0.022*"visually_stunne" + 0.020*"highly" + 0.017*"head" + '
  '0.016*"alone" + 0.015*"basis" + 0.015*"screenplay" + 0.015*"aside" + '
  '0.007*"logic" + 0.007*"detract"'),
 (1,
  '0.028*"man" + 0.015*"master" + 0.014*"event" + 0.013*"cinematic" + '
  '0.013*"product" + 0.011*"heavy" + 0.010*"ancient" + 0.009*"evil" + '
  '0.009*"death" + 0.009*"sean"'),
 (2,
  '0.015*"place" + 0.015*"perhaps" + 0.014*"rather" + 0.014*"dragon" + '
  '0.013*"easily" + 0.013*"ride" + 0.012*"conflict" +

# 3.)

We generated 10 topics.\
Top 10 words from each topic are as follows:\
Topic 0: "stand", "visually_stunne", "highly", "head", "alone", "basis", "screenplay", "aside", "logic", "detract"\
Topic 1: "man", "master", "event", "cinematic", "product", "heavy", "ancient", "evil", "death", "sean"\
Topic 2: "place", "perhaps", "rather", "dragon", "easily", "ride", "conflict", "opinion", "genre", "mythology"\
Topic 3: "movie", "marvel", "good", "great", "action", "story", "scene", "film", "fight", "watch"\
Topic 4: "disappointed", "pick", "american", "coaster", "roller", "judge", "snake", "hiphoprap", "accurate", "confirm"\
Topic 5: "film", "character", "shangchi", "new", "also", "asian", "s", "ring", "first", "leave"\
Topic 6: "cliche", "sure", "aspect", "return", "issue", "less", "generic", "skill", "acting", "talk"\
Topic 7: "force", "sequel", "hidden_dragon", "diverse", "dark", "masterpiece", "ragnarok", "appreciate", "crouching_tiger", "value"\
Topic 8: "sight", "service", "apart", "streaming", "smash", "park", "sensation", "span", "sensory", "stigma"\
Topc 9: "fine", "clear", "absolute", "spoiler", "requirement", "top_notch", "notice", "creative", "dude, "context"\

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.458231736424015

Coherence Score:  0.45974788434588376


In [ ]:
import pyLDAvis.gensim_models
#pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\maganti\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.369586  0.035715       1        1  62.092371
5      0.281956 -0.050487       2        1  25.341984
2     -0.105851 -0.266966       3        1   4.604378
6     -0.100846  0.204343       4        1   3.973787
1     -0.102280  0.035375       5        1   1.423612
9     -0.078588  0.014174       6        1   1.095768
7     -0.078379  0.009285       7        1   0.812829
0     -0.064215  0.006867       8        1   0.284228
4     -0.063786  0.006702       9        1   0.263366
8     -0.057598  0.004993      10        1   0.107677, topic_info=           Term         Freq        Total Category  logprob  loglift
38        movie  1641.000000  1641.000000  Default  30.0000  30.0000
72         good   692.000000   692.000000  Default  29.0000  29.0000
127        film   623.000000   623.000000  Default  28.0000  28.0000
47     shangchi   171.000000   17

# **Question 2: Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. Note: Cross-validation (5-fold or 10-fold) should be conducted. Here is the reference of cross-validation: https://scikit-learn.org/stable/modules/cross_validation.html.

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9. 

In [ ]:
# Write your code here

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')

data = pd.read_csv("/content/gdrive/My Drive/data.csv")

data['clean_text'] = data['clean_text'].str.lower()
data['sentiment'] = data['sentiment'].str.lower()

# getting stopwords of english
stopwords = stopwords.words("english")
stopwords = [re.sub('[^a-zA-Z0-9]+', '', word) for word in stopwords]

# removing all special characters
data['clean_text'] = data['clean_text'].str.replace('[^a-zA-Z0-9 ]', '')
data['clean_text'] = data['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

# applying lemmatization on the tweets
data['clean_text'] = data['clean_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="v") for word in row.split()))
data['clean_text'] = data['clean_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="n") for word in row.split()))
data['clean_text'] = data['clean_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="a") for word in row.split()))
data['clean_text'] = data['clean_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="r") for word in row.split()))
data.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maganti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\maganti\AppData\Local\Temp/ipykernel_30708/3113960571.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_text'] = data['clean_text'].str.replace('[^a-zA-Z0-9 ]', '')
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maganti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,document_id,clean_text,sentiment
0,0,amaze movie marvel studio hype movie since fir...,positive
1,1,ill honest marvel movie top even within marvel...,positive
2,2,finally fun entertain movie without garbage ho...,negative
3,3,goal mcu solo film sell audience new character...,negative
4,4,seem one marvel late marvel studio offer movie...,neutral


In [ ]:
data['sentiment'].value_counts()

positive    864
negative     58
neutral      20
Name: sentiment, dtype: int64

# 1.)

In [ ]:
data['sentiment'] = data['sentiment'].replace({"positive": 0, 'negative': 1, "neutral": 2})
 
X = data['clean_text']
Y = data['sentiment']

# initializing tf-idf vectorizer
vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1, 2))

# fitting the train data and generating the features
vectorizer = vectorizer.fit(X)

# transforming the data using vectorizer model that is fit using train data 
X_features = vectorizer.transform(X).toarray()

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_features, Y, test_size=0.2)


# 2.)

# Cross-validation on Decision Tree classifier

In [ ]:
from sklearn.model_selection import cross_val_score
dt_clr = DecisionTreeClassifier(max_depth=5)
scores = cross_val_score(dt_clr, X_features, Y, cv=5)
print(scores)

[0.88359788 0.87301587 0.89893617 0.90425532 0.90957447]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

dt_clr = DecisionTreeClassifier(max_depth=5)
dt_clr.fit(X_train, y_train)
y_pred = dt_clr.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.98      0.92       166
           1       0.00      0.00      0.00        19
           2       0.00      0.00      0.00         4

    accuracy                           0.86       189
   macro avg       0.29      0.33      0.31       189
weighted avg       0.77      0.86      0.81       189



C:\Users\maganti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maganti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maganti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Cross-validation on Random Forest classifier

In [ ]:
from sklearn.model_selection import cross_val_score
rf_clr = RandomForestClassifier(max_depth=2, random_state=0)
scores = cross_val_score(rf_clr, X_features, Y, cv=5)
print(scores)

[0.91534392 0.91534392 0.92021277 0.92021277 0.91489362]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clr = RandomForestClassifier(max_depth=2, random_state=0)
rf_clr.fit(X_train, y_train)
y_pred = rf_clr.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94       166
           1       0.00      0.00      0.00        19
           2       0.00      0.00      0.00         4

    accuracy                           0.88       189
   macro avg       0.29      0.33      0.31       189
weighted avg       0.77      0.88      0.82       189



C:\Users\maganti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maganti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maganti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 3.)

. The metrics for classification are precision, recall, accuracy, f1-score.\
. The decision tree accuracy is less than random forest accuracy.\
. Since the dataset is imbalanced, the f1 scores of negative(1) and neutral(2) classes are 0.\
. Among the two classifiers above, random forest classifier is the best sentiment classifier.

# **Question 3: House price prediction**

(40 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download here: https://github.com/unt-iialab/info5731-spring2022/blob/main/assignments/assignment4-question3-data.zip. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878. 


In [ ]:
# Write your code here

import pandas as pd
import numpy as np

# load the train dataset
train_data = pd.read_csv("/content/gdrive/My Drive/assignment4-question3-data/train.csv")

train_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
# null values percentage
for column in train_data.columns:
    if train_data[column].isna().sum()/train_data.shape[0]*100 != 0:
        print(column, "", train_data[column].isna().sum(), "   ", round(train_data[column].isna().sum()/train_data.shape[0]*100, 2))


LotFrontage  259     17.74
Alley  1369     93.77
MasVnrType  8     0.55
MasVnrArea  8     0.55
BsmtQual  37     2.53
BsmtCond  37     2.53
BsmtExposure  38     2.6
BsmtFinType1  37     2.53
BsmtFinType2  38     2.6
Electrical  1     0.07
FireplaceQu  690     47.26
GarageType  81     5.55
GarageYrBlt  81     5.55
GarageFinish  81     5.55
GarageQual  81     5.55
GarageCond  81     5.55
PoolQC  1453     99.52
Fence  1179     80.75
MiscFeature  1406     96.3


In [ ]:
# dropping columns with higher null values % and columns that have year values.
train_data = train_data.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'YearBuilt',
                             'YearRemodAdd', 'GarageYrBlt', 'YrSold'], axis = 1)


In [ ]:
for column in train_data.columns:
    if train_data[column].isna().sum()/train_data.shape[0]*100 != 0:
        print((train_data[column]).dtype, column , "", train_data[column].isna().sum(), "   ", round(train_data[column].isna().sum()/train_data.shape[0]*100, 2))
#print(train_data.isna().sum())


float64 LotFrontage  259     17.74
object MasVnrType  8     0.55
float64 MasVnrArea  8     0.55
object BsmtQual  37     2.53
object BsmtCond  37     2.53
object BsmtExposure  38     2.6
object BsmtFinType1  37     2.53
object BsmtFinType2  38     2.6
object Electrical  1     0.07
object GarageType  81     5.55
object GarageFinish  81     5.55
object GarageQual  81     5.55
object GarageCond  81     5.55


In [ ]:
# replacing null values with mean, mode values of that column.
for col in ['LotFrontage', 'MasVnrArea']:
    train_data[col].fillna(train_data[col].mean(), inplace = True)
    
for col in ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 
            'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_data[col].fillna(train_data[col].mode()[0], inplace = True)


In [ ]:
for column in train_data.columns:
    if train_data[column].isna().sum()/train_data.shape[0]*100 != 0:
        print((train_data[column]).dtype, column , "", train_data[column].isna().sum(), "   ", round(train_data[column].isna().sum()/train_data.shape[0]*100, 2))


In [ ]:
print(train_data.shape)

# categorical columns dummification 
train_data = pd.get_dummies(train_data)
print(train_data.shape)

(1460, 72)
(1460, 268)


# Linear regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

X = train_data.drop(['SalePrice'], axis = 1)
y = train_data['SalePrice']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_valid)

print("RMSE:", np.sqrt(mean_squared_error(y_valid, y_pred)))
print("R2 Score:", r2_score(y_valid, y_pred))

RMSE: 54720.020379664515
R2 Score: 0.5664138888104355


# Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtreg = DecisionTreeRegressor(max_depth=5)
dtreg.fit(X_train, y_train)
y_pred = dtreg.predict(X_valid)

print("RMSE:", np.sqrt(mean_squared_error(y_valid, y_pred)))
print("R2 Score:", r2_score(y_valid, y_pred))

RMSE: 46456.33468816109
R2 Score: 0.6874836488038527


# Random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(max_depth=5, random_state=0)
rf_reg.fit(X_train, y_train)
y_pred = rf_reg.predict(X_valid)

print("RMSE:", np.sqrt(mean_squared_error(y_valid, y_pred)))
print("R2 Score:", r2_score(y_valid, y_pred))

RMSE: 37255.72132814937
R2 Score: 0.7990126083080934


In [ ]:
test_data = pd.read_csv("/content/gdrive/My Drive/assignment4-question3-data/test.csv")

test_data = test_data.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'YearBuilt',
                             'YearRemodAdd', 'GarageYrBlt', 'YrSold'], axis = 1)

for column in test_data.columns:
    if test_data[column].isna().sum()/test_data.shape[0]*100 != 0:
        print((test_data[column]).dtype, column, "", test_data[column].isna().sum(), "   ", round(test_data[column].isna().sum()/test_data.shape[0]*100, 2))
#print(train_data.isna().sum())


object MSZoning  4     0.27
float64 LotFrontage  227     15.56
object Utilities  2     0.14
object Exterior1st  1     0.07
object Exterior2nd  1     0.07
object MasVnrType  16     1.1
float64 MasVnrArea  15     1.03
object BsmtQual  44     3.02
object BsmtCond  45     3.08
object BsmtExposure  44     3.02
object BsmtFinType1  42     2.88
float64 BsmtFinSF1  1     0.07
object BsmtFinType2  42     2.88
float64 BsmtFinSF2  1     0.07
float64 BsmtUnfSF  1     0.07
float64 TotalBsmtSF  1     0.07
float64 BsmtFullBath  2     0.14
float64 BsmtHalfBath  2     0.14
object KitchenQual  1     0.07
object Functional  2     0.14
object GarageType  76     5.21
object GarageFinish  78     5.35
float64 GarageCars  1     0.07
float64 GarageArea  1     0.07
object GarageQual  78     5.35
object GarageCond  78     5.35
object SaleType  1     0.07


In [ ]:
# replacing null values with mean, mode values of that column.
for col in ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
           'BsmtHalfBath', 'GarageCars', 'GarageArea']:
    test_data[col].fillna(test_data[col].mean(), inplace = True)
    
for col in ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
           'BsmtFinType1', 'BsmtFinType2', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual',
           'GarageCond', 'SaleType']:
    test_data[col].fillna(test_data[col].mode()[0], inplace = True)


In [ ]:
for column in test_data.columns:
    if test_data[column].isna().sum()/test_data.shape[0]*100 != 0:
        print((test_data[column]).dtype, column, "", test_data[column].isna().sum(), "   ", round(test_data[column].isna().sum()/test_data.shape[0]*100, 2))
#print(train_data.isna().sum())

In [ ]:
print(test_data.shape)

test_data = pd.get_dummies(test_data)
print(test_data.shape)

(1459, 71)
(1459, 251)


In [ ]:
for col in train_data.columns:
    if col not in test_data.columns and col != 'SalePrice':
        print(col)
        test_data[col] = 0

Utilities_NoSeWa
Condition2_RRAe
Condition2_RRAn
Condition2_RRNn
HouseStyle_2.5Fin
RoofMatl_ClyTile
RoofMatl_Membran
RoofMatl_Metal
RoofMatl_Roll
Exterior1st_ImStucc
Exterior1st_Stone
Exterior2nd_Other
Heating_Floor
Heating_OthW
Electrical_Mix
GarageQual_Ex


In [ ]:
print(test_data.shape)

(1459, 267)


In [ ]:
# Among all the models, random forest regressor performs well
# as the R2 score is better.
y_pred = rf_reg.predict(test_data)

C:\Users\maganti\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [ ]:
# predictions for test data
y_pred

array([116524.52992537, 149149.14958385, 182709.17281549, ...,
       149007.60848061, 108780.85592607, 226827.64341166])